In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np


In [2]:
pre_process_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4' 

In [4]:
import tensorflow_hub as hub
%pip install tensorflow_hub

bert_model = hub.load(encoder_url)



ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\blaik\AppData\Local\Temp\tfhub_modules\d760773f85f64fc84ae0b47310f7cfe3bcec4868' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

spam detection

In [ ]:
spam_data_raw = pd.read_csv('spam.csv')

In [ ]:
spam_data_raw.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
spam_data_raw.Category.unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
encoded_spam_column = pd.get_dummies(spam_data_raw.Category).drop(['ham'],axis = 1,)

In [ ]:
display(encoded_spam_column)

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [ ]:
spam_data_raw['spam'] =  encoded_spam_column

In [ ]:
spam_data_raw

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [ ]:
spam_data_raw.to_csv('spam_preprocessed.csv')

In [ ]:
preprocessed_text = bert_preprocess_model(spam_data_raw.Message)

In [ ]:
preprocessed_text.keys()

dict_keys(['input_type_ids', 'input_word_ids', 'input_mask'])

In [ ]:
preprocessed_text['input_word_ids']

<tf.Tensor: shape=(5572, 128), dtype=int32, numpy=
array([[  101,  2175,  2127, ...,     0,     0,     0],
       [  101,  7929,  2474, ...,     0,     0,     0],
       [  101,  2489,  4443, ...,     0,     0,     0],
       ...,
       [  101, 12063,  1010, ...,     0,     0,     0],
       [  101,  1996,  3124, ...,     0,     0,     0],
       [  101, 20996, 10258, ...,     0,     0,     0]])>

In [ ]:
embedding_result = bert_model(preprocessed_text)

In [ ]:
embedding_result.keys()

In [ ]:
input = tf.keras.layers.Input(shape=(128))
hidden_layer1 = tf.keras.layers.Dense(64,activation= 'sigmoid')(input)
hidden_drop1 = tf.keras.layers.Dropout(0.2)(hidden_layer1)
hidden_layer2 = tf.keras.layers.Dense(128,activation= 'sigmoid')(hidden_drop1)
hidden_drop2 = tf.keras.layers.Dropout(0.5)(hidden_layer2)
hidden_layer3 = tf.keras.layers.Dense(64,activation= 'sigmoid')(hidden_drop2)
hidden_drop3 = tf.keras.layers.Dropout(0.2)(hidden_layer3)
output = tf.keras.layers.Dense(10,activation= 'softmax')(hidden_drop3)



In [ ]:
funct_model = tf.keras.models.Model(inputs = input, outputs = output)

In [ ]:
funct_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 128)               8320      
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_8 (Dropout)         (None, 64)                0   

In [ ]:
funct_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

x_train,y_train, x_test, y_yest = train_test_split()